## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,64.99,63,57,4.00,broken clouds
1,1,Vaini,TO,-21.20,-175.20,80.60,74,20,13.87,few clouds
2,2,Madera,US,37.17,-119.83,66.00,41,0,4.18,clear sky
3,3,Rikitea,PF,-23.12,-134.97,72.68,80,23,18.79,few clouds
4,4,Ushuaia,AR,-54.80,-68.30,51.80,50,75,13.87,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.20,-175.20,80.60,74,20,13.87,few clouds
10,10,Hilo,US,19.73,-155.09,84.20,66,40,9.17,scattered clouds
11,11,Nisia Floresta,BR,-6.09,-35.21,78.80,83,40,9.17,scattered clouds
13,13,Airai,TL,-8.93,125.41,82.31,32,97,5.48,overcast clouds
21,21,Flinders,AU,-34.58,150.86,80.01,59,73,4.00,broken clouds
22,22,Broken Hill,AU,-31.95,141.43,77.00,47,75,14.99,broken clouds
27,27,Kavieng,PG,-2.57,150.80,85.87,69,100,4.83,light rain
30,30,Tual,ID,-5.67,132.75,81.36,80,93,13.62,overcast clouds
37,37,Cabo San Lucas,MX,22.89,-109.91,78.01,83,5,10.29,clear sky
41,41,Butaritari,KI,3.07,172.79,81.70,72,88,15.77,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.20,-175.20,80.60,74,20,13.87,few clouds
10,10,Hilo,US,19.73,-155.09,84.20,66,40,9.17,scattered clouds
11,11,Nisia Floresta,BR,-6.09,-35.21,78.80,83,40,9.17,scattered clouds
13,13,Airai,TL,-8.93,125.41,82.31,32,97,5.48,overcast clouds
21,21,Flinders,AU,-34.58,150.86,80.01,59,73,4.00,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,80.60,few clouds,-21.20,-175.20,
10,Hilo,US,84.20,scattered clouds,19.73,-155.09,
11,Nisia Floresta,BR,78.80,scattered clouds,-6.09,-35.21,
13,Airai,TL,82.31,overcast clouds,-8.93,125.41,
21,Flinders,AU,80.01,broken clouds,-34.58,150.86,
22,Broken Hill,AU,77.00,broken clouds,-31.95,141.43,
27,Kavieng,PG,85.87,light rain,-2.57,150.80,
30,Tual,ID,81.36,overcast clouds,-5.67,132.75,
37,Cabo San Lucas,MX,78.01,clear sky,22.89,-109.91,
41,Butaritari,KI,81.70,overcast clouds,3.07,172.79,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,80.60,few clouds,-21.20,-175.20,Keleti Beach Resort
10,Hilo,US,84.20,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
11,Nisia Floresta,BR,78.80,scattered clouds,-6.09,-35.21,Rancho Crepusculo
13,Airai,TL,82.31,overcast clouds,-8.93,125.41,Juvinal Martin
21,Flinders,AU,80.01,broken clouds,-34.58,150.86,Shellharbour Resort & Conference Centre
22,Broken Hill,AU,77.00,broken clouds,-31.95,141.43,Daydream Motel & Cottages
27,Kavieng,PG,85.87,light rain,-2.57,150.80,Nusa Island Retreat
30,Tual,ID,81.36,overcast clouds,-5.67,132.75,Grand Vilia Hotel
37,Cabo San Lucas,MX,78.01,clear sky,22.89,-109.91,Hotel Tesoro Los Cabos
41,Butaritari,KI,81.70,overcast clouds,3.07,172.79,Isles Sunset Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 25.0), zoom_level=1.1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))